In [1]:
import os
from omegaconf import OmegaConf
from nuplan.common.actor_state.vehicle_parameters import get_pacifica_parameters
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.scenario_builder.abstract_scenario import AbstractScenario
from nuplan.planning.simulation.controller.abstract_controller import AbstractEgoController
from nuplan.planning.simulation.observation.tracks_observation import TracksObservation
from nuplan.planning.simulation.planner.abstract_planner import AbstractPlanner
from nuplan.planning.simulation.simulation_time_controller.step_simulation_time_controller import (
    StepSimulationTimeController,
)
from nuplan.planning.simulation.simulation import Simulation
from nuplan.planning.simulation.simulation_setup import SimulationSetup
from nuplan.planning.script.builders.worker_pool_builder import build_worker
from nuplan.planning.simulation.controller.perfect_tracking import PerfectTrackingController

NUPLAN_DATA_ROOT = os.getenv('NUPLAN_DATA_ROOT', '/data/sets/nuplan')
NUPLAN_MAPS_ROOT = os.getenv('NUPLAN_MAPS_ROOT', '/data/sets/nuplan/maps')
NUPLAN_DB_FILES = os.getenv('NUPLAN_DB_FILES', '/data/sets/nuplan/nuplan-v1.1/splits/mini')
NUPLAN_MAP_VERSION = os.getenv('NUPLAN_MAP_VERSION', 'nuplan-maps-v1.0')
NUPLAN_SENSOR_ROOT = f"{NUPLAN_DATA_ROOT}/nuplan-v1.1/sensor_blobs"
DB_FILE = f"{NUPLAN_DATA_ROOT}/nuplan-v1.1/splits/mini/2021.05.12.22.28.35_veh-35_00620_01164.db"
MAP_NAME = "us-nv-las-vegas"

In [29]:
scenario_builder = NuPlanScenarioBuilder(
    data_root=NUPLAN_DATA_ROOT,
    map_root=NUPLAN_MAPS_ROOT,
    sensor_root=NUPLAN_SENSOR_ROOT,
    db_files=[DB_FILE],
    map_version=NUPLAN_MAP_VERSION,
    vehicle_parameters=get_pacifica_parameters(),
    include_cameras=False,
    verbose=True
)

scenario_filter = ScenarioFilter(
    log_names = ["2021.05.12.22.28.35_veh-35_00620_01164"],
    scenario_types = None,
    scenario_tokens = None,
    map_names = None,
    num_scenarios_per_type = None,
    limit_total_scenarios = None,
    timestamp_threshold_s = None,
    ego_displacement_minimum_m = None,
    expand_scenarios = False,
    remove_invalid_goals = False,
    shuffle = False
)

worker_config = OmegaConf.create({
    'worker': {
        '_target_': 'nuplan.planning.utils.multithreading.worker_sequential.Sequential',  # Specify the worker class
        # 'num_workers': 4,  # Number of workers
        # '_target_': "nuplan.planning.utils.multithreading.worker_parallel.SingleMachineParallelExecutor",  # Specify the worker class
        # '_target_': 'nuplan.planning.script.builders.worker_pool.Sequental',  # Specify the worker class
    }
})

worker = build_worker(worker_config)
scenarios = scenario_builder.get_scenarios(scenario_filter, worker)

# Print the number of scenarios
print("Number of scenarios: ", len(scenarios))
scenario = scenarios[0]  # Get the first scenario
# print the type of the scenario
print("Scenario type: ", type(scenario))

# print("Number of scenarios: ", len(scenario))

time_controller = StepSimulationTimeController(scenario)
observation = TracksObservation(scenario)
# observations = build_observations(observation, scenario=scenario)
controller = PerfectTrackingController(scenario)

simulation_setup = SimulationSetup(
    time_controller=time_controller,  # Required simulation time controller
    observations=observation,  # Required observation
    ego_controller=controller,  # Required ego controller
    scenario=scenario  # Required scenario
)

simulation = Simulation(
    simulation_setup=simulation_setup,  # Required simulation setup
    callback=None,  # Optional callback functions
    simulation_history_buffer_duration=2.0  # Optional history buffer duration
)

simulation.initialize()

print("NuPlan initialized.")

Number of scenarios:  9600
Scenario type:  <class 'nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario.NuPlanScenario'>
NuPlan initialized.


In [11]:
planner_input = simulation.get_planner_input()

# print all the attributes of the planner input
print("\nPlanner attributes:")
for attr in dir(planner_input):
    if not attr.startswith('__'):
        print(attr)

# history is of type SimulationHistoryBuffer
history = planner_input.history
# print all the attributes of the history

print("\nHistory attributes:")
for attr in dir(history):
    if not attr.startswith('__'):
        print(attr)

# iteration is of type SimulationIteration
iteration = planner_input.iteration

# print all the attributes of the iteration that do not start with '__'
print("\nIteration attributes:")
for attr in dir(iteration):
    if not attr.startswith('__'):
        print(attr)


Planner attributes:
history
iteration
traffic_light_data

History attributes:
_ego_state_buffer
_observations_buffer
_sample_interval
append
current_state
duration
ego_state_buffer
ego_states
extend
initialize_from_list
initialize_from_scenario
observation_buffer
observations
sample_interval
size

Iteration attributes:
index
time_point
time_s
time_us


In [27]:
last_ego_state = history.ego_state_buffer[-10]
ego_state, observation_state = history.current_state

last_ego_position = last_ego_state.waypoint.center
ego_postition = ego_state.waypoint.center

agents = observation_state.tracked_objects.get_agents()
first_agent = agents[0]

print("Last ego position: " + str(last_ego_position))
print("Ego position: " + str(ego_postition))

print("Agent position: " + str(first_agent.center))

Last ego position: StateSE2(x=664431.8721475152, y=3998295.4482887965, heading=-1.521316690939634)
Ego position: StateSE2(x=664431.9236408463, y=3998294.590372303, heading=-1.5198627795173558)
Agent position: StateSE2(x=664422.5570504119, y=3998309.3846771335, heading=1.733215541888221)
